In [1]:
import xarray as xr
xr.set_options(keep_attrs=True,
               display_expand_data=False)
import pandas as pd

import plotly.express as px
import plotly.io as pio
# pio.renderers.default='png'

from stats import get_factor_data


In [2]:
factor_data = get_factor_data()
data = factor_data
factor_master = pd.DataFrame(data.asset.attrs).T


[*********************100%%**********************]  30 of 30 completed


In [3]:
# halflifes = [21, 63, 126, 252, 512]
# data = get_factor_data(halflifes)
# factor_master = pd.DataFrame(data['asset'].attrs).T
# factor_list = factor_master.index.to_list()

# MDS

In [119]:
v = df_t.loc['SPY']
x, y = v
arctan2(y, x)

-1.1333374765051418

In [ ]:
x, y = df_t.loc['SPY']
theta = arctan2(y, x)
r1= rotation(-theta)
x1, y1 = r1 @ array([x, y])

# sqrt(x**2+y**2) - sqrt(x1**2+y1**2)
x, y
x1, y1

In [158]:
(r1 @ df_t.T).T

,0,1
asset,,
SPY,0.638114,-7.820584e-17
IWM,0.298362,-3.589551e-01
QQQ,0.656443,2.077044e-01
DIA,0.613836,-2.560699e-01
FEZ,0.429938,1.493217e-01
EWU,0.464347,-1.218204e-01
DAX,0.334980,1.534224e-01
EWC,0.320096,-2.095597e-01
CAC,0.192006,-6.422764e-01


In [ ]:
def transform_coordinates(coordinates: pd.DataFrame, transformation_type=None, factor: str = 'SPY'):
    
    
    def rotation(theta):
        """
        Create a 2D rotation matrix for a given angle theta.

        Parameters
        ----------
        theta : float
            The angle of rotation in radians.

        Returns
        -------
        np.ndarray
            The 2D rotation matrix.
        """
        return array([[cos(theta), -sin(theta)],
                      [sin(theta), cos(theta)]])
        
    
    
    def rotate(coordinates, theta):
        rotation_matrix = rotation(-theta)
        transformed_coordinates = (rotation_matrix @ coordinates.T).T
        transformed_coordinates.columns = coordinates.columns
        return transformed_coordinates #.rename(columns=dict(enumerate(df_t.columns)))
    
    # def scale(coordinates, length):
    #     return coordinates / length
    
    if transformation_type is None:
        return coordinates
    
    v = df_t.loc[factor]
    x, y = v
        
    if transformation_type == 'rotate':
        # theta = arctan2(*v) #- np.pi/2
        theta = arctan2(y, x)
        return coordinates.pipe(rotate, -theta)
    
    if transformation_type == 'normalize':
        # theta = arctan2(*v) #- np.pi/2
        theta = arctan2(y, x)
        length = sqrt(sum(v**2))
        return coordinates.pipe(rotate, -theta).div(length)


In [58]:
data.corr.sel(date=slice(None, None)).date.values

array(['1965-01-05T00:00:00.000000000', '1965-01-06T00:00:00.000000000',
       '1965-01-07T00:00:00.000000000', ...,
       '2024-11-05T00:00:00.000000000', '2024-11-06T00:00:00.000000000',
       '2024-11-07T00:00:00.000000000'], dtype='datetime64[ns]')

In [ ]:
from sklearn.manifold import MDS
from numpy import sqrt

def multidimensional_scaling(correlation_matrix: pd.DataFrame, init=None) -> pd.DataFrame:
    """
    Perform multidimensional scaling on a correlation matrix.

    Parameters
    ----------
    correlation_matrix : pd.DataFrame
        The input correlation matrix as a pandas DataFrame.
    init : np.ndarray, optional
        Initial positions of the points in the embedding space. If None, random initialization is used.

    Returns
    -------
    pd.DataFrame
        A DataFrame containing the MDS results with dimensions 'dim1' and 'dim2'.
    """
    
    dissimilarity_matrix = sqrt(1 - correlation_matrix**2)

    # Pass n_init explicitly to suppress warning when init is not None:
    n_init = 4 if init is None else 1 
    embedding = MDS(dissimilarity='precomputed', random_state=42, n_init=n_init)
    coordinates = embedding.fit_transform(dissimilarity_matrix, init=init)
    
    return pd.DataFrame(coordinates, 
                        index=dissimilarity_matrix.index, 
                        columns=pd.Index(['dim1', 'dim2'], name='dimension'))


def mds_ts_df(corr: xr.DataArray, start_date = None, transformation=None, factor='SPY') -> pd.DataFrame:
    dates = data.corr.sel(date=slice('2024', None)).date.values
    coordinates = None
    
    mds_dict = {}
    for date in dates:
        df = corr.sel(date=date, corr_type=63).to_pandas()
        coordinates = multidimensional_scaling(df, init=coordinates)
        transformed = transform_coordinates(coordinates, transformation, factor=factor)
        mds_dict[date] = transformed
    return (pd.concat(mds_dict)
            .rename_axis(index=['date', df.index.name])
            # .stack()
            # .rename('mds')
            # .to_xarray()
            )


# corr_df = data.corr.sel(date=data.date.max(), corr_type=63).to_pandas()
# df = multidimensional_scaling(corr_df)


def draw_mds(df: pd.DataFrame) -> px.scatter:
    # df.plot.scatter(x='dim1', y='dim2')
    fig_format = {'template': 'plotly_white', 'height': 750, 'width': 750}
    fig = (px.scatter(df.join(factor_master).reset_index(), 
                      x='dim1', y='dim2', text='asset', color='asset_class',
                     **fig_format)
           .update_traces(textposition='middle right', textfont=dict(color='lightgray'))
           .update_xaxes(title=None)
           .update_yaxes(title=None))
    return fig


def draw_mds_ts(df: pd.DataFrame) -> px.scatter:
    # df.plot.scatter(x='dim1', y='dim2')
    fig_format = {'template': 'plotly_white', 'height': 750, 'width': 750}
    
    fig = (px.scatter(df, 
                      x='dim1', y='dim2', text='asset', color='asset_class', 
                      animation_frame='date',
                     **fig_format)
           .update_traces(textposition='middle right', textfont=dict(color='lightgray'))
           .update_xaxes(title=None)
           .update_yaxes(title=None))
    return fig

# mds_series = mds_da.to_series().reset_index().join(factor_master, on='asset')
# draw_mds_ts(mds_series)
# mds_series

# mds_da.to_series().reset_index().pivot(index='date', columns='dimension')

# mds_df

In [9]:
import numpy as np

def euclidean_length(series: pd.Series) -> float:
    """
    Calculate the Euclidean length (norm) of a pandas Series.

    Parameters
    ----------
    series : pd.Series
        The input pandas Series.

    Returns
    -------
    float
        The Euclidean length of the Series.
    """
    return np.sqrt(np.sum(series**2))

# Example usage:
example_series = pd.Series([3, 4])
length = euclidean_length(example_series)
print(length)  # Output: 5.0

5.0


In [154]:
x, y = df_t.loc['SPY']
theta = arctan2(y, x)
r1= rotation(-theta)
x1, y1 = r1 @ array([x, y])

# sqrt(x**2+y**2) - sqrt(x1**2+y1**2)
x, y
x1, y1

(0.6381142328017737, -7.367828420636568e-17)

In [140]:
# Calculate the angle to rotate
angle = arctan2(y, x)

# Create the rotation matrix
rotation_matrix = array([[cos(-angle), -sin(-angle)],
                         [sin(-angle), cos(-angle)]])

# Rotate the vector
rotated_vector = rotation_matrix @ array([x, y])

# Normalize the rotated vector to align with [1, 0]
rotated_vector = rotated_vector / sqrt(rotated_vector[0]**2 + rotated_vector[1]**2)

print(rotated_vector)  # Output should be close to [1, 0]


[ 1.00000000e+00 -1.15462531e-16]


In [126]:
transform_coordinates(df_t) #, transformation_type='rotate')

dimension,dim1,dim2
asset,,
SPY,0.270330,-0.578024
IWM,-0.198755,-0.422333
QQQ,0.466240,-0.506635
DIA,0.028089,-0.664513
FEZ,0.317399,-0.326193
EWU,0.086367,-0.472228
DAX,0.280886,-0.238440
EWC,-0.054221,-0.378731
CAC,-0.500452,-0.446019


In [160]:
df = mds_ts_df(data.corr, transformation='rotate', start_date='2024')
df1 = df.reset_index().join(factor_master, on='asset')
df1['date'] = df1['date'].astype(str)
draw_mds_ts(df1)

In [104]:
df_t = df.xs(df.index[0][0])
v = df_t.loc['SPY']
x = v.iloc[0]
y = v.iloc[1]
angle = arctan2(y, x)
rotate_to_x_axis(angle)

TypeError: cannot unpack non-iterable numpy.float64 object

In [44]:
# draw_mds(df_t).show()
# draw_mds(transform_coordinates(df_t)).show()
# df_t
draw_mds(transform_coordinates(df_t, 'rotate')).show()
draw_mds(transform_coordinates(df_t, 'normalize')).show()

In [ ]:
draw_mds(df.xs(df.index[0][0]))

In [ ]:
break

# Scratch

In [ ]:
type(fig)

In [ ]:
data.corr.sel(date=date, corr_type=63).to_pandas().pipe(multidimensional_scaling)

In [ ]:
# mds_ts = {date: data.corr.sel(date=date, corr_type=63).to_pandas().pipe(multidimensional_scaling)
#           for date in data.corr.date}
# for date in data.corr.date:
data.corr.date

In [ ]:
from sklearn.manifold import MDS
from numpy import sqrt


correlation_matrix = ds.corr.sel(date='2024-10-25', corr_type=63).to_pandas()
dissimilarity_matrix = sqrt(1 - correlation_matrix**2)


mds = MDS(dissimilarity='precomputed', random_state=42)
mds_results = mds.fit_transform(dissimilarity_matrix)
mds_df = pd.DataFrame(mds_results, index=dissimilarity_matrix.index, columns=['dim1', 'dim2'])


print(mds_df)
mds_df.plot.scatter(x='dim1', y='dim2')

px.scatter(mds_df.reset_index(), x='dim1', y='dim2', hover_name='asset')

In [ ]:
from sklearn.manifold import MDS

corr_df = ds.corr.sel(date='2024-10-25', corr_type=63).to_pandas()
mds = MDS(dissimilarity='precomputed', random_state=42)
mds_results = mds.fit_transform(1 - corr_df)
mds_df = pd.DataFrame(mds_results, index=corr_df.index, columns=['MDS1', 'MDS2'])
print(mds_df)
mds_df.plot.scatter(x='MDS1', y='MDS2')

px.scatter(mds_df.reset_index(), x='MDS1', y='MDS2', hover_name='asset')

In [ ]:
mds_results_list = []

for date in ds.date.values:
    corr_df = ds.corr.sel(date=date, corr_type=63).to_pandas()
    imputer = SimpleImputer(strategy='mean')
    corr_df_imputed = imputer.fit_transform(1 - corr_df)
    mds = MDS(dissimilarity='precomputed', random_state=42)
    mds_results = mds.fit_transform(corr_df_imputed)
    mds_results = mds.fit_transform(1 - corr_df)
    mds_results_list.append(mds_results)

mds_results_array = np.stack(mds_results_list)
mds_da = xr.DataArray(mds_results_array, coords=[ds.date, corr_df.index, ['MDS1', 'MDS2']], dims=['date', 'asset', 'mds_dim'])
print(mds_da)


# Scratch 2

In [ ]:
def rotate_to_unit_x(v):
    """
    Rotate a vector v = [x, y] to align with the unit vector [1, 0].

    Parameters
    ----------
    v : list or np.ndarray
        The input vector [x, y].

    Returns
    -------
    np.ndarray
        The rotated vector aligned with [1, 0].
    """
    x, y = v
    angle = arctan2(y, x)
    rotation_matrix = array([[cos(-angle), -sin(-angle)],
                             [sin(-angle), cos(-angle)]])
    rotated_vector = rotation_matrix @ array([x, y])
    return rotated_vector / sqrt(rotated_vector[0]**2 + rotated_vector[1]**2)

# Example usage:
v = [3, 4]
rotated_v = rotate_to_unit_x(v)
print(rotated_v)  # Output should be close to [1, 0]

In [80]:
# Example usage:
v = [3, 4]
rotated_v = rotate_to_x_axis(v)
print(rotated_v)


[5.00000000e+00 5.55111512e-16]


In [108]:
def rotate_to_unit_x(v):
    """
    Rotate a vector v = [x, y] to align with the unit vector [1, 0].

    Parameters
    ----------
    v : list or np.ndarray
        The input vector [x, y].

    Returns
    -------
    np.ndarray
        The rotated vector aligned with [1, 0].
    """
    x, y = v
    angle = arctan2(y, x)
    rotation_matrix = array([[cos(-angle), -sin(-angle)],
                             [sin(-angle), cos(-angle)]])
    rotated_vector = rotation_matrix @ array([x, y])
    return rotated_vector / sqrt(rotated_vector[0]**2 + rotated_vector[1]**2)

# Example usage:
v = [3, 4]
rotated_v = rotate_to_unit_x(v)
print(rotated_v)  # Output should be close to [1, 0]

[1.00000000e+00 1.11022302e-16]


In [ ]:
import numpy as np
from numpy import sin, cos, arctan2, array


def rotate_to_x_axis(v):
    """
    Rotate a vector v = [x, y] to lie on the x-axis.

    Parameters
    ----------
    v : list or np.ndarray
        The input vector [x, y].

    Returns
    -------
    np.ndarray
        The rotated vector lying on the x-axis.
    """
    x, y = v
    angle = arctan2(y, x)
    rotation_matrix = array([[cos(angle), -sin(angle)],
                             [sin(angle), cos(angle)]])
    rotated_vector = rotation_matrix @ array([x, y])
    return rotated_vector


# Example usage:
v = [3, 4]
rotated_v = rotate_to_x_axis(v)
print(rotated_v)





def transform_mds_gpt(coordinates: pd.DataFrame, factor: str = 'SPY'):
    """
    Transform the MDS coordinates to align a given factor with the x-axis.

    Parameters
    ----------
    coordinates : pd.DataFrame
        The MDS coordinates as a DataFrame with columns 'dim1' and 'dim2'.
    factor : str, optional
        The factor to align with the x-axis. Default is 'SPY'.

    Returns
    -------
    pd.DataFrame
        The transformed MDS coordinates with the factor aligned to the x-axis.
    """
    # Get the coordinates of the factor
    factor_coordinates = coordinates.loc[factor]
    
    # Rotate the MDS coordinates to align the factor with the x-axis
    angle = arctan2(*factor_coordinates[::-1])
    rotation_matrix = rotation(-angle)
    transformed_coordinates = coordinates.apply(lambda x: rotation_matrix @ x, axis=1)
    
    return transformed_coordinates
    


[-1.4  4.8]
